<a href="https://colab.research.google.com/github/Deepsphere-AI/DSAI_Python_Programming/blob/main/Unit-16/Python%20for%20Deep%20Learning/Program%20342%20-%20CSLAB_DEEP_LEARNING_NORMALIZING_CREATING_SEQUENCES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:

# /*********************************************************************************************************************
  
#   File Name 	:   CSLAB_DEEP_LEARNING_NORMALIZING_CREATING_SEQUENCES
#   Purpose 	:   A Program in Python for Normalizing & Creating Sequences using Deep Learning
#   Author	:   Deepsphere.ai
#   Reviewer 	:   Jothi Periasamy
#   Date :   26/10/2022
#   Version	:   1.0	

# /***********************************************************************************************************************

## Program Description : Program for Normalizing & Creating Sequences using Deep Learning in Python

## Python Development Environment & Runtime - Python, Anaconda

import pandas as vAR_pd

import tensorflow as vAR_tf  

from google.colab import drive

drive.mount('/content/drive')

vAR_CSLAB_CREATE_SEQUENCES = '/content/drive/MyDrive/Semester-4/Deep Learning for Enterprise/Data/crypto_data/LTC-USD.csv'

vAR_CSLAB_df = vAR_pd.read_csv(vAR_CSLAB_CREATE_SEQUENCES, names=['time', 'low', 'high', 'open', 'close', 'volume'])

#print(vAR_CSLAB_df.head())

vAR_CSLAB_main_df = vAR_pd.DataFrame() # begin empty

ratios = ["BTC-USD", "LTC-USD", "BCH-USD", "ETH-USD"]  # the 4 ratios we want to consider
for ratio in ratios:  # begin iteration
    #print(ratio)
    vAR_CSLAB_dataset = "/content/drive/MyDrive/Semester-4/Deep Learning for Enterprise/Data/crypto_data/BTC-USD.csv"  # get the full path to the file.
    vAR_CSLAB_df = vAR_pd.read_csv(vAR_CSLAB_dataset, names=['time', 'low', 'high', 'open', 'close', 'volume'])  # read in specific file

    # rename volume and close to include the ticker so we can still which close/volume is which:
    vAR_CSLAB_df.rename(columns={"close": f"{ratio}_close", "volume": f"{ratio}_volume"}, inplace=True)

    vAR_CSLAB_df.set_index("time", inplace=True)  # set time as index so we can join them on this shared time
    vAR_CSLAB_df = vAR_CSLAB_df[[f"{ratio}_close", f"{ratio}_volume"]]  # ignore the other columns besides price and volume

    if len(vAR_CSLAB_main_df)==0:  # if the dataframe is empty
        vAR_CSLAB_main_df = vAR_CSLAB_df  # then it's just the current df
    else:  # otherwise, join this data to the main one
        vAR_CSLAB_main_df = vAR_CSLAB_main_df.join(vAR_CSLAB_df)
        
vAR_CSLAB_main_df.fillna(method="ffill", inplace=True)  # if there are gaps in data, use previously known values
vAR_CSLAB_main_df.dropna(inplace=True)
#print(vAR_CSLAB_main_df.head())  # how did we do??

vAR_CSLAB_SEQ_LEN = 60  # how long of a preceeding sequence to collect for RNN
vAR_CSLAB_FUTURE_PERIOD_PREDICT = 3  # how far into the future are we trying to predict?
vAR_CSLAB_RATIO_TO_PREDICT = "LTC-USD"

def classify(current, future):
    if float(future) > float(current):
        return 1
    else:
        return 0
    
vAR_CSLAB_main_df['future'] = vAR_CSLAB_main_df[f'{vAR_CSLAB_RATIO_TO_PREDICT}_close'].shift(-vAR_CSLAB_FUTURE_PERIOD_PREDICT)

vAR_CSLAB_main_df['target'] = list(map(classify, vAR_CSLAB_main_df[f'{vAR_CSLAB_RATIO_TO_PREDICT}_close'], vAR_CSLAB_main_df['future']))

#print(vAR_CSLAB_main_df.head())

vAR_CSLAB_x_train = vAR_CSLAB_main_df[['BTC-USD_close','BTC-USD_volume','LTC-USD_close', 'LTC-USD_volume']]

vAR_CSLAB_y_train = vAR_CSLAB_main_df['target']

vAR_CSLAB_model = vAR_tf.keras.models.Sequential()  

# takes our 28x28 and makes it 1x784

vAR_CSLAB_model.add(vAR_tf.keras.layers.Flatten())  

vAR_CSLAB_model.add(vAR_tf.keras.layers.Dense(128, activation=vAR_tf.nn.relu))  

vAR_CSLAB_model.add(vAR_tf.keras.layers.Dense(128, activation=vAR_tf.nn.relu))  

vAR_CSLAB_model.add(vAR_tf.keras.layers.Dense(10, activation=vAR_tf.nn.softmax))  

vAR_CSLAB_model.compile(optimizer='adam',  # Good default optimizer to start with
              loss='sparse_categorical_crossentropy',  # how will we calculate our "error." Neural network aims to minimize loss.
              metrics=['accuracy'])  # what to track

vAR_CSLAB_model.fit(vAR_CSLAB_x_train, vAR_CSLAB_y_train, epochs=3)  # train the model

vAR_CSLAB_val_loss, vAR_CSLAB_val_acc = vAR_CSLAB_model.evaluate(vAR_CSLAB_x_train, vAR_CSLAB_y_train)  # evaluate the out of sample data with model

print(vAR_CSLAB_val_loss)  # model's loss (error)

print(vAR_CSLAB_val_acc)  # model's accuracy

# /****************************************************************************************************************************
#   Disclaimer.

# We are providing this code block strictly for learning and researching, this is not a production
# ready code. We have no liability on this particular code under any circumstances; users should use
# this code on their own risk. All software, hardware and othr products that are referenced in these 
# materials belong to the respective vendor who developed or who owns this product.

# /****************************************************************************************************************************
  

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Epoch 1/3
3054/3054 [==============================] - 9s 2ms/step - loss: 22.0284 - accuracy: 0.5039
Epoch 2/3
3054/3054 [==============================] - 8s 2ms/step - loss: 5.8362 - accuracy: 0.5056
Epoch 3/3
3054/3054 [==============================] - 5s 2ms/step - loss: 2.0798 - accuracy: 0.4490
2.0797927379608154
0.4490094482898712


**Code Explanation:**

This code is written in Python and uses the libraries pandas and google.colab.

The first line import pandas as vAR_pd imports the pandas library and renames it to vAR_pd for convenience. The next line from google.colab import drive imports the drive module from the google.colab library.

The line drive.mount('/content/drive') mounts the Google Drive in the Colab environment so that the code can access files stored on the drive.

Next, the variable vAR_CSLAB_CREATE_SEQUENCES is defined as a string representing the file path to a .csv file stored on the Google Drive. This file contains the data for the cryptocurrency Litecoin (LTC) in USD.

The line vAR_CSLAB_df = vAR_pd.read_csv(vAR_CSLAB_CREATE_SEQUENCES, names=['time', 'low', 'high', 'open', 'close', 'volume']) uses the read_csv method of pandas to read the data from the file into a DataFrame object, vAR_CSLAB_df. The names parameter specifies the names of the columns in the data.

The line vAR_CSLAB_main_df = vAR_pd.DataFrame() creates an empty DataFrame object called vAR_CSLAB_main_df.

The line ratios = ["BTC-USD", "LTC-USD", "BCH-USD", "ETH-USD"] creates a list of strings that represent the different cryptocurrencies and their prices in USD.

The for loop that follows starts the iteration over the different cryptocurrencies. In each iteration, the code reads the data for the cryptocurrency and its price in USD from the Google Drive and stores it in a DataFrame object called vAR_CSLAB_df. The columns 'close' and 'volume' are then renamed to include the ticker name, so that it is possible to keep track of which 'close' and 'volume' data belongs to which cryptocurrency.

The index of the DataFrame is set to the 'time' column using the set_index method, so that the data from different cryptocurrencies can be joined based on the shared time.

The code then checks if vAR_CSLAB_main_df is empty. If it is, then vAR_CSLAB_df is assigned to vAR_CSLAB_main_df. If it is not, then the join method is used to join vAR_CSLAB_df with vAR_CSLAB_main_df. This process is repeated for all the cryptocurrencies in the list ratios

The line vAR_CSLAB_main_df.fillna(method="ffill", inplace=True) uses the fillna method to fill any missing values in the DataFrame with the previously known value. The line vAR_CSLAB_main_df.dropna(inplace=True) drops any remaining rows with missing values.

The line vAR_CSLAB_SEQ_LEN = 60 sets the variable vAR_CSLAB_SEQ_LEN to 60, which represents the number of preceding time steps